In [2]:
import json
import pandas as pd
import glob
import os
import ast

In [3]:
##list all files
path_name = glob.glob(os.path.join("/Users/yuweiwang/RA/ie-evaluation/metadata", "*.metadata"))
path_name.sort()

In [4]:
#load files
files = [open(f,"r") for f in path_name]
lines = [file.readlines() for file in files]

In [5]:
#choose only topic == 'agent/dialog'
def getExtractions(line_file):
    dialog_data = []
    j_file = [json.loads(line) for line in line_file]
    for i in range(len(j_file)):
        if j_file[i]['topic'] == "agent/dialog":
            if j_file[i]['data']['text'][0] != '{':
                dialog_data.append([j_file[i]['data']['text'].strip(),j_file[i]['data']['extractions']])
    return dialog_data

In [6]:
#get text and extractions and save to data
dataAll = []
for file in lines:
    dataAll += getExtractions(file)

In [7]:
len(dataAll)

6821

In [8]:
#save dataAll into csv for future use
pd.DataFrame(dataAll, columns= ["utt", "extractions"]).to_csv("metadata2extraction.csv", index=False)

In [ ]:
# if the runtime of above is too long, load the saved output.
dataAll_df = pd.read_csv("metadata2extraction.csv",index_col=None, header=0)
data_utt = dataAll_df['utt'].to_list()
data_extractions = dataAll_df['extractions'].to_list()
clean_extractions = [ast.literal_eval(i) for i in data_extractions]
dataAll = list(zip(data_utt,clean_extractions))

In [9]:
#get only the labels of the extractions
data_labels = []
for i in dataAll:
    data_labels.append([i[0].strip(),[extraction['labels'] for extraction in i[1]]])

In [24]:
labels = {'0':0,
"CriticalVictim":1,
'Victim':2,
'Room':3,
'Engineer':4,
'Transporter':5,
'Medic':6,
'Rubble':7,
'MarkerBlock':8,
'Meeting':9,
'Move':10,
'Precedence':11,
'RescueInteractions':12,
'KnowledgeSharing':13,
'ReportLocation':14,
'Search':15,
'HelpRequest':16,
'Question':17,
'YesNoQuestion':18,
'Instruction':19,
'Plan':20
}

In [37]:
#select 20 labels and tokenize them
labels20 = []
for i in data_labels:
    found = []
    tokens = []
    if i[1]:
        overlaps = []
        for j in i[1]:
            overlap = set(labels) & set(j)
            if len(overlap) > 1 and "CriticalVictim" in overlap:
                overlap = {'CriticalVictim'}
            elif len(overlap) > 1 and "YesNoQuestion" in overlap:
                overlap = {'YesNoQuestion'}
            overlaps += overlap
            if overlap:
                found += overlap
                tokens += [labels[l] for l in overlap]
        if not overlaps:
            found.append('0')
            tokens.append(0)
    else:
        found.append('0')
        tokens.append(0)
    labels20.append([i[0], found, tokens])

In [30]:
count = 0
for i in labels20:
    if "Victim" in i[1]:
        count += 1
count

589

In [40]:
pd.DataFrame(labels20,columns=["utt", "labels", "tokenized"]).to_csv("metadata2tokenzied.csv", index=False)